# File Downloads

In [1]:
#from google.cloud import storage

#def download_file(bucket_name = 'train_data_solar_project',
#                              blob_name = 'X_train_copernicus.npz', 
#                              download_to_disk = False, 
#                              destination_file_name = '../raw_data/data.npz'):
#    
#    """Download a file from Google Cloud Storage. 
#    If download_to_disk = False then it will save to memory.  
#    If download_to_disk = True then it will save to your local disk.
#    """
#    
#    storage_client = storage.Client()
#
#    bucket = storage_client.bucket(bucket_name)
#
#    blob = bucket.blob(blob_name)
#    
#    if download_to_disk == True:
#       
#        blob.download_to_filename(destination_file_name)
#        print(
#            "Downloaded storage object {} from bucket {} to local file {}.".format(
#            blob_name, bucket_name, destination_file_name
#        )
#    )
#
#   if download_to_disk == False:
#
#        contents = blob.download_as_string()
#
#        print("Downloaded storage object {} from bucket {} as the following string: {}.".format(
#            blob_name, bucket_name, contents
#        )
#    )
#        
#    return 'Download complete.'

# File Exploration

In [2]:
import numpy as np
import datetime

def number_of_observations(actual_day):
    
    '''Takes a date and returns the number of observations taken that day.'''
    
    dx = datetime.datetime.strptime(actual_day, "%d-%m-%Y")
    dx = dx.date()

    d1 = datetime.date(2012,1,1)
    d2 = datetime.date(2012,2,20)
    d3 = datetime.date(2012,4,12)
    d4 = datetime.date(2012,8,28)
    d5 = datetime.date(2012,10,20)
    d6 = datetime.date(2012,12,31)
    
    obs = 0
    
    if dx>=d1 and dx<=d2:
        obs=4    
    elif dx>d2 and dx<=d3:
        obs=5
    elif dx>d3 and dx<=d4:
        obs=6
    elif dx>d4 and dx<=d5:
        obs=5
    elif dx>d5 and dx<=d6:
        obs=4

    if obs!=0:
        return obs
    else:
        return ("The date out of range.")

In [3]:
number_of_observations('21-06-2012')

6

In [4]:
def date_index(actual_day):
    
    '''Takes a date and returns the list of relevant indices in the data.'''
    
    data = np.load('../raw_data/X_train.npz', allow_pickle = True) # Figure out to make this a python script
    riv = data['datetime']
    date_format = []
    
    for i in range(riv.shape[0]):
        date_format.append(riv[i].date()) 
    
    dx = datetime.datetime.strptime(actual_day, "%d-%m-%Y")
    dx = dx.date()
   
    index = []
    
    for i in range(riv.shape[0]):
        if dx == date_format[i]:
            index.append(i)
    
    if not index:
        return ("The date out of range.")
    else:
        return index

In [5]:
# Finding the index for the last day of Janurary for model-building purposes
date_index('31-01-2012')

[120, 121, 122, 123]

In [67]:
# defining time variable 
def complete_timestamp_day(observation= date_index('21-06-2012')[0]):
    
    data = np.load('../raw_data/X_train.npz', allow_pickle = True) # Figure out to make this a python script
    DATE = data['datetime']
    # converting specific date into a str
    _ = DATE[observation]
    date_str = _.strftime('%d-%m-%Y')
    
    arr_time_day = []
    
    time = datetime.datetime(year=DATE[observation].year,\
                                    month=DATE[observation].month,
                                    day=DATE[observation].day,
                                    hour=DATE[observation].hour,
                                    minute=DATE[observation].minute)
    
    delta = datetime.timedelta(minutes=15)
    time = time - datetime.timedelta(minutes=60)
    # we're setting the num of obs in day",
    observation_max = number_of_observations(date_str)
    for j in range(observation_max):
        for i in range(8):
            time = time+delta
            arr_time_day.append(time)
    
    arr_time_day  = np.array(arr_time_day)
    return arr_time_day

In [69]:
complete_timestamp_day(observation= date_index('31-01-2012')[0])

array([datetime.datetime(2012, 1, 31, 8, 15),
       datetime.datetime(2012, 1, 31, 8, 30),
       datetime.datetime(2012, 1, 31, 8, 45),
       datetime.datetime(2012, 1, 31, 9, 0),
       datetime.datetime(2012, 1, 31, 9, 15),
       datetime.datetime(2012, 1, 31, 9, 30),
       datetime.datetime(2012, 1, 31, 9, 45),
       datetime.datetime(2012, 1, 31, 10, 0),
       datetime.datetime(2012, 1, 31, 10, 15),
       datetime.datetime(2012, 1, 31, 10, 30),
       datetime.datetime(2012, 1, 31, 10, 45),
       datetime.datetime(2012, 1, 31, 11, 0),
       datetime.datetime(2012, 1, 31, 11, 15),
       datetime.datetime(2012, 1, 31, 11, 30),
       datetime.datetime(2012, 1, 31, 11, 45),
       datetime.datetime(2012, 1, 31, 12, 0),
       datetime.datetime(2012, 1, 31, 12, 15),
       datetime.datetime(2012, 1, 31, 12, 30),
       datetime.datetime(2012, 1, 31, 12, 45),
       datetime.datetime(2012, 1, 31, 13, 0),
       datetime.datetime(2012, 1, 31, 13, 15),
       datetime.datetime(

In [8]:
# defining solar elevation as per Federico definition 
def solar_elevation(
    feature, 
    observation = date_index('21-06-2012')[0]): 
    arr_sol_elev_day = []
    
    # we're setting the num of obs in day\n",
    observation_max = number_of_observations('21-06-2012')
    
    for j in range(observation_max):
        for i in range(8):
            arr_sol_elev_day.append(90-(round((feature[observation+j,i,:,:]).mean(),2)))
        
    arr_sol_elev_day = np.array(arr_sol_elev_day)
    return arr_sol_elev_day

In [9]:
def cloudy_sky_condition(
    feature,
    observation = date_index('21-06-2012')[0]): 
    arr_day = []
    
    # we're setting the num of obs in day\n",
    observation_max = number_of_observations('21-06-2012')
    
    for j in range(observation_max):
        for i in range(8):
            if i<4:
                arr_day.append(round((feature[observation+j,i,:,:]).mean(),2))
            if i>3:
                arr_day.append(0)
              
    arr_day = np.array(arr_day)
    return arr_day

In [10]:
def clear_sky_condition(
    feature,
    observation = date_index('21-06-2012')[0]): 
    arr_day = []
    
    # we're setting the num of obs in day\n",
    observation_max = number_of_observations('21-06-2012')
    
    for j in range(observation_max):
        for i in range(8):
            arr_day.append(round((feature[observation+j,i,:,:]).mean(),2))
    arr_day = np.array(arr_day)
    return arr_day

In [11]:
def solar_azimuth_angle(
    feature,
    observation = date_index('21-06-2012')[0]): 
    arr_day = []
    
    # we're setting the num of obs in day\n",
    observation_max = number_of_observations('21-06-2012')
    
    for j in range(observation_max):
        for i in range(8):
            arr_day.append(round((feature[observation+j,i,:,:]).mean(),2))
    arr_day = np.array(arr_day)
    return arr_day

In [12]:
# def day_information(
#         feature = SAA,
#         observation = 96,
#         timestamp = 0
#         ):
#         # Print total radiation for all pixels in a timestamp
#         #print(f’The radiation on {date[observation]} was {feature[observation, timestamp, :, :].sum()}.’)
#         #print(f’The radiation on {date[observation]} looks like:’)
#         time = datetime.datetime(year = date[observation].year,
#                                  month=date[observation].month,
#                                  day = date[observation].day,
#                                  hour=date[observation].hour,
#                                  minute=date[observation].minute)
#         delta=datetime.timedelta(minutes=15)
#         time = time - datetime.timedelta(minutes=60)
#         # we’re setting the num of obs in day,
#         observation_max = n_obs_in_day(observation)
#         # Define number of rows and columns you want in your figure
#         nrow = observation_max
#         ncol = 8
#         fig = plt.figure(figsize = (100,60))
#         gs = GridSpec(nrow, ncol + 1, figure=fig)
#         # Fill your figure with desired plots
#         axes = []
#         for j in range(observation_max):
#             for i in range(8):
#                 time=time+delta
#                 #plt.subplot(6, 8, j * 8 + i + 1)
#                 axes.append(fig.add_subplot(gs[j, i]))
#                 plt.title(time.time(), fontsize=80)
#                 plt.xticks(fontsize=35)\
#                 plt.yticks(fontsize=35)
#                 im = axes[-1].imshow(feature[observation+j,i,:,:],
#                            cmap=‘jet’, norm=‘linear’, vmin = min_max(feature)[0],vmax= min_max(feature)[1])
#         month = date[observation].month
#         fig.suptitle(f’Solar Azimuth Angle on {date[observation].day} of {calendar.month_name[month]}‘,
#                      fontsize=110)
#         # Shared colorbar method 1”,
#         ax9 = fig.add_subplot(gs[:, 8])
#         axes.append(ax9)
#         cbar = fig.colorbar(im, cax=axes[-1],)
#         tick_font_size = 90
#         cbar.ax.tick_params(labelsize=tick_font_size)
#         # we should use a variable to describe the feature
#         # got an error when tried
#         cbar.ax.set_title(f’SAA Range’,fontsize=80)

In [13]:
# ax9 = fig.add_subplot(gs[:, 8])
# cbar = fig.colorbar(im, cax=axes[-1],)
# tick_font_size = 90
# cbar.ax.tick_params(labelsize=tick_font_size)
# cbar.ax.set_title(f’SAA Range’,fontsize=80)

# GHI and CLS Evolution

In [14]:
# import numpy as np
# import datetime
# import matplotlib.pyplot as plt
# from mpl_toolkits.axes_grid1 import make_axes_locatable

# def one_day_evolution(
#     feature = 'CLS',
#     date = '21-06-2012'):
    
#     '''Takes date and time as input to returns the visual evolution of selected feature over that day.'''    

#     data = np.load('../raw_data/X_train.npz', allow_pickle = True)
#     DATE = data['datetime']
    
#     observations = number_of_observations(date)
#     selected_feature = data[feature]
#     index = date_index(date)
#     first_index = date_index(date)[0]
#     time = DATE[first_index] 
#     time = time - datetime.timedelta(minutes=60)
#     delta=datetime.timedelta(minutes=15)
        
#     vmin=(selected_feature[first_index:first_index+observations,:,:,:]).min()
#     vmax=(selected_feature[first_index:first_index+observations,:,:,:]).max()
        
#     X = [[0,0,81,81]]
        
#     plt.figure (figsize = (150,100))
#     plt.title(f"Displaying the full day on {date}", fontsize=100)

#     if feature == 'CLS':
#         for j in range(observations):
#             for i in range(8):
#                 time=time+delta
#                 ax = plt.subplot(observations, 8, j * 8 + i + 1)
#                 plt.title(time.time(), fontsize=100)
#                 pic = ax.imshow(selected_feature[first_index+j,i,:,:], cmap='jet', vmin=vmin, vmax=vmax)
    
#     elif feature == 'GHI':
#         for j in range(observations):
#             for i in range(8):
#                 time=time+delta
#                 ax = plt.subplot(observations, 8, j * 8 + i + 1)
#                 plt.title(time.time(), fontsize=100)
#                 if i > 3:
#                     pic = ax.imshow(X, cmap='binary', vmin=0, vmax=0)
#                     plt.xlim(200)
#                     plt.ylim(200)

#                 else:
#                     pic = ax.imshow(selected_feature[first_index + j, i, :, :], cmap='jet', vmin=vmin, vmax=vmax)

#     plt.colorbar(pic)
#     print(f'The radiation on {date} was {selected_feature[index[0] + j, :, :, :].sum()} W/m\u00b2.')


In [329]:
import matplotlib.gridspec as gridspec
import numpy as np
import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

def one_day_evolution(
    feature = 'CLS',
    date = '21-06-2012'):
    
    '''Takes date and time as input to returns the visual evolution of selected feature over that day.'''    

    data = np.load('../raw_data/X_train.npz', allow_pickle = True)
    DATE = data['datetime']
    
    observations = number_of_observations(date)
    selected_feature = data[feature]
    index = date_index(date)
    first_index = date_index(date)[0]
    time = DATE[first_index] 
    time = time - datetime.timedelta(minutes=60)
    delta=datetime.timedelta(minutes=15)
        
    vmin=(selected_feature[first_index:first_index+observations,:,:,:]).min()
    vmax=(selected_feature[first_index:first_index+observations,:,:,:]).max()
        
    X = [[0,0,81,81]]
    
    # Define number of rows and columns you want in your figure
    nrow = observations
    ncol = 8
  
    # Fill your figure with desired plots
    axes = []
    fig = plt.figure (figsize = (160,110),constrained_layout = True)
    fig.tight_layout(pad=5.0)
    plt.title(f"Displaying the full day on {date}", fontsize=130, y=1.02)
    gs = gridspec.GridSpec(nrow, ncol + 1, figure=fig)

    if feature == 'CLS':
        for j in range(observations):
            for i in range(8):
                time=time+delta
                #ax = plt.subplot(observations, 8, j * 8 + i + 1)
                axes.append(fig.add_subplot(gs[j, i],aspect=20))
                #plt.title(time.time(), fontsize=100)
                pic = axes[-1].imshow(selected_feature[first_index+j,i,:,:], cmap='jet', vmin=vmin, vmax=vmax)
                axes[-1].set_title(time.time(), fontsize=120)
                
                
        # Shared colorbar method 1”,
        ax9 = fig.add_subplot(gs[:, 8])
        axes.append(ax9)
        cbar = fig.colorbar(pic, cax=axes[-1],shrink = 0.5)
        tick_font_size = 90
        cbar.ax.tick_params(labelsize=tick_font_size)

    elif feature == 'GHI':
        for j in range(observations):
            for i in range(8):
                time=time+delta
                #ax = plt.subplot(observations, 8, j * 8 + i + 1)
                axes.append(fig.add_subplot(gs[j, i]))
                #plt.title(time.time(), fontsize=100)
                if i > 3:
                    pic = axes[-1].imshow(X, cmap='binary', vmin=0, vmax=0)
                    plt.xlim(200)
                    plt.ylim(200)

                else:
                    pic1 = axes[-1].imshow(selected_feature[first_index + j, i, :, :], cmap='jet', vmin=vmin, vmax=vmax)
                    axes[-1].set_title(time.time(), fontsize=120)
        
        # Shared colorbar method 1”,
        ax9 = fig.add_subplot(gs[:, 8])
        axes.append(ax9)
        cbar = fig.colorbar(pic1, cax=axes[-1],shrink = 0.5)
        tick_font_size = 90
        cbar.ax.tick_params(labelsize=tick_font_size)
        
    

In [326]:
one_day_evolution('CLS', '26-02-2012')

In [330]:
one_day_evolution('GHI', '26-02-2012')

# Model Functions

In [17]:
def add_dumb_layers(observation):
    '''Takes a timestamp and duplicates four times for comeplete GHI datasets''' 
    
    X_train = np.load('../raw_data/X_train_copernicus.npz')
    
    new_layer = np.expand_dims(X_train['GHI'][observation][3], axis = 0)
    selected_layer = X_train['GHI'][observation]
    
    for i in range(4):
        selected_layer = np.vstack((selected_layer, new_layer))
        return selected_layer[4:8, :, :]